<a href="https://www.kaggle.com/code/mh0386/inception-network?scriptVersionId=239917340" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
from os import listdir

from kagglehub import dataset_download
from keras import Model
from keras.src.applications.inception_v3 import InceptionV3
from keras.src.layers import Dense, GlobalAveragePooling2D
from keras.src.utils import load_img, to_categorical
from matplotlib import pyplot
from numpy import argmax, asarray, zeros
from PIL import Image
from seaborn import heatmap
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

In [ ]:
dataset_path = dataset_download("abhinavnayak/catsvdogs-transformed")
print("Path to dataset files:", dataset_path)

In [ ]:
data_path: str = f"{dataset_path}/train_transformed/"
print(data_path)

In [ ]:
list_of_images = listdir(data_path)
number_of_images: int = len(list_of_images)
print(number_of_images)

In [ ]:
i = 1
pyplot.figure(figsize=(10, 10))
for image in list_of_images:
    if i > 5:
        break
    img = load_img(data_path + image)
    pyplot.subplot(1, 5, i)
    pyplot.imshow(img)
    pyplot.title(str(img.size))
    pyplot.axis("off")
    i += 1

pyplot.show()

In [ ]:
images = zeros((number_of_images, 224, 224, 3))
labels = zeros((number_of_images, 1))

In [ ]:
for image_number, image_path in enumerate(list_of_images):
    img = Image.open(data_path + image_path)
    img = asarray(img)
    img = img / img.max()
    images[image_number] = img
    labels[image_number] = 0 if "cat" in image_path else 1

In [ ]:
print(labels)

In [ ]:
labels = to_categorical(labels)

In [ ]:
print(labels)

In [ ]:
images.shape

In [ ]:
pyplot.imshow(images[0])

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(
    images, labels, test_size=0.2, random_state=42
)

In [ ]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

In [ ]:
x_train, x_test = x_train / 255.0, x_test / 255.0

In [ ]:
model = InceptionV3(
    include_top=False,
    weights="imagenet",
    input_shape=(224, 224, 3),
)

In [ ]:
# Freeze the pre-trained layers
for layer in model.layers:
    layer.trainable = False

In [ ]:
# Add layers using calls:
x = model.output  # Get the output of the InceptionV3 base
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation="relu")(x)
predictions = Dense(2, activation="softmax")(x)

In [ ]:
# Create a new Functional model with these layers:
model = Model(inputs=model.input, outputs=predictions)

In [ ]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
history = model.fit(x_train, y_train, epochs=50)

In [ ]:
pyplot.plot(history.history["accuracy"])
pyplot.plot(history.history["loss"])
pyplot.title("Model Accuracy")
pyplot.ylabel("Accuracy")
pyplot.xlabel("Epoch")
pyplot.show()

In [ ]:
model.evaluate(x_test, y_test)

In [ ]:
y_predict = model.predict(x_test)

In [ ]:
y_predict = argmax(y_predict, axis=1)
y_test_argmax = argmax(y_test, axis=1)

In [ ]:
cm = confusion_matrix(y_test_argmax, y_predict)
heatmap(cm, annot=True, fmt="d")

In [ ]:
cr = classification_report(y_test_argmax, y_predict)
print(cr)